In [ ]:
# install backtesting package
!pip install backtesting

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

# .NS suffix is required for National Stock Exchange of India
ticker = "HDFCBANK.NS"
data = yf.download(ticker, start="2018-01-01", end="2024-12-20", multi_level_index=False)

# Standardize columns for Backtesting library
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data.ffill(inplace=True)

In [ ]:
# --- Strategy 1: SMA Crossover (The "Golden Cross") ---
class Bank_SMA_Cross(Strategy):
    n1 = 20  # Fast (Monthly average)
    n2 = 50  # Slow (Quarterly average)

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()

# --- Strategy 2: Relative Strength Momentum ---
class Bank_Momentum(Strategy):
    lookback = 60  # 3 months of trading days

    def init(self):
        # Calculate rate of change over 60 days
        self.mom = self.I(lambda x: pd.Series(x).pct_change(periods=self.lookback), self.data.Close)

    def next(self):
        # Buy if 3-month momentum is positive
        if self.mom[-1] > 0.05 and not self.position:  # Added 5% threshold for conviction
            self.buy()
        # Exit if momentum fades below 0
        elif self.mom[-1] < 0 and self.position:
            self.position.close()

In [ ]:
# Run Backtests
bt_sma = Backtest(data, Bank_SMA_Cross, cash=100000, commission=0.0012, finalize_trades=True)
stats_sma = bt_sma.run()

bt_mom = Backtest(data, Bank_Momentum, cash=100000, commission=0.0012, finalize_trades=True)
stats_mom = bt_mom.run()

# Build Comparison Table
comparison = pd.DataFrame({"SMA Crossover": stats_sma, "Momentum": stats_mom})
metrics = ["Return [%]", "Buy & Hold Return [%]", "Sharpe Ratio", "Max. Drawdown [%]", "# Trades"]

print("--- HDFC Bank Strategy Comparison ---")
print(comparison.loc[metrics])

In [ ]:
# To visualize the SMA strategy:
# bt_sma.plot()

# To visualize the Momentum strategy:
bt_mom.plot()